# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np
from scipy import optimize
from scipy.optimize import minimize, minimize_scalar
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import *

## Robinson Crusoe Economy Model

We consider a simplified economy consisting of only one inhabitant, Robinson Crusoe, living in a world where only two goods exist: $x_1$ and $x_2$.

#### Utility Function

Robinson derives utility from these goods through the following Cobb-Douglas utility function:

$$
U(x_1, x_2) = x_1^\alpha x_2^{1-\alpha}
$$

where $0 < \alpha < 1$. This parameter, $\alpha$, represents Robinson's relative preference for $x_1$ over $x_2$, highlighting the substitutability and relative importance of each good in satisfying his utility.

#### Production Function

Robinson can produce $x_2$ by converting $x_1$ through a production process described by:

$$
f(z) = z^\beta
$$

where $z$ represents the quantity of $x_1$ allocated to the production of $x_2$, and $0 < \beta < 1$ indicates diminishing returns to scale in the production of $x_2$. This production function encapsulates the physical transformation of $x_1$ into $x_2$ with efficiency determined by $\beta$.

### Model Solution

To solve this model, we aim to find an optimal allocation of goods $(x_1, x_2)$ such that no other feasible allocation yields higher utility for Robinson. The constraints of the model are defined by Robinson’s total endowments and his production capabilities, which can be expressed as:

$$
x_1 = e_1 - z
$$

$$
x_2 = e_2 + f(z)
$$

Here, \(e_1\) and \(e_2\) are Robinson's initial endowments of goods \(x_1\) and \(x_2\) respectively. The goal is to determine the value of \(z\) that maximizes Robinson's utility given his initial endowments and the production technology available to him, hence solving the following maximization problem:

$$
\max_{x_1,x_2}{u(x_1,x_2)} \space \space u.c. \space \space x_1 = e_1 - z, \space \space x_2 = e_2 + f(z)
$$

As preferences are assumed to be convex and production is concave, the problem can be solved by

$$
MRS = TRS
$$

For $0<\beta<1$, there is no algebraic solution for this problem, hence we proceed to a numerical approach in the following


## Numerical solution

We start out by solving the simple model generating random endowments, preference and production efficiency.

In [71]:
# Set seed
np.random.seed(0)

# Generating parameters
alpha = np.random.random()
beta = np.random.random()
e1 = np.random.random()
e2 = np.random.random()

# Utility function
def u(x1, x2, alpha):
    return x1**alpha * x2**(1-alpha)

# Production function
def f(z, beta):
    return z**beta

# Function to minimize (-utility)
def objective_no_trade(x1, alpha, beta, e1, e2):
    z = e1 - x1
    x2 = e2 + f(z, beta)
    return -u(x1, x2, alpha)

# x1 must be between 0 and e1
bounds = (0, e1)

# Perform the minimization
result = minimize_scalar(objective_no_trade, bounds=bounds, method='bounded', args=(alpha, beta, e1, e2))

# Maximizing values
x1_max = result.x
z_max = e1 - x1_max
x2_max = e2 + f(z_max, beta)
u_max = u(x1_max, x2_max, alpha)

print(f"Robinson Crusoe: Optimal x1: {x1_max:.2f}, Optimal x2: {x2_max:.2f}, Optimal u: {u_max:.2f} ")

Robinson Crusoe: Optimal x1: 0.54, Optimal x2: 0.69, Optimal u: 0.60 


Next we proceed by adding more people to the economy. We assume that all inhabitants have similar utility and production functions, but their preference, efficiency and endowments vary. Hence, we generate a random $\alpha$, $\beta$, $e_1$ and $e_2$ for each inhabitant.

In [ ]:
# Number of people
N = 10

# Set seed
np.random.seed(0)

# Parameters
alphas = np.random.random()
betas = np.random.random()
e1 = np.random.random() * 10
e2 = np.random.random() * 10

We start out by deriving each habitant's initial utility based on endowments

Next, we allow people in the economy to trade. We find Wallrasian equilibrium and derive each habitant's utility.

Next, we allow disallow people to trade, but instead allow them to produce.

Finally, we also allow for people to produce and derive habitant's utility in Wallrasian equilibrium

# Conclusion

In conclusion, we find that both allowing for trade and production seperately allows for pareto improvements due to an expansion of each inhabitant's consumption possibility frontier. Same conlcusions hold when allowing for trade in the economy, where only habitants 